In [0]:
import sys
sys.path.append("../../")

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from src_staging.sv_policy_stage import create_policy_stage, link_policy_agent_schema

In [0]:
partition_name = 'partition_date'
partition_date = '20250828'

In [0]:
_bz_table = 'ctl_central_published.sc_bz_source_name.dt_customer_journey_daily'
_link_sv_table = 'ctl_central_published.sc_sv_raw_vault.link_policy_agent'
partition_date_fmt = f"{partition_date[:4]}-{partition_date[4:6]}-{partition_date[6:]}"

In [0]:
sv_df = create_policy_stage(_bz_table, partition_name, partition_date_fmt)
sv_df.display()

In [0]:
if not spark.catalog.tableExists(_link_sv_table):
    (spark.createDataFrame([], link_policy_agent_schema())
         .write.format("delta")
         .saveAsTable(_link_sv_table))

In [0]:
todays_links = sv_df.select(
                    "policy_number",
                    "license_number",
                    "record_source",
                    "partition_date"
                )

In [0]:
existing_keys = spark.table(_link_sv_table).select("policy_number","license_number")

new_links = (todays_links
    .join(existing_keys, on=["policy_number","license_number"], how="left_anti").dropDuplicates()
    .withColumn("_ingest_ts", current_timestamp())
    .select("policy_number","license_number","record_source","_ingest_ts","partition_date")
)

In [0]:
new_links.write.format("delta").mode("append").saveAsTable(_link_sv_table)